# KNN Music Classfication

In [2]:
#import libraries
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

In [3]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [4]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

* trainingset is actual training data and k is hyperparameter of how much nearest neighbors we want.
* first part itera gets distances of alll the points in training set
* we sort the data and get the nearest neighbor, as the data is sorted so it will easily take the k nearest neighbors. and we finally return those neighbors

In [5]:
#function to identify the nearest neighbors
def nearestclass(neighbors):
    classVote = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

* here we define a set, which will help to take the reponse of each class present in nearest neighbors.
* actually the idea behind this function is that, we already have a list of nearest neighbors and as data is labelled data so each neighbor will hold some particular class, now from that nearest neighbor whatever the maximum frequency will be there for each class, to that class the new data will be assigned.
* eg- k=10, so will have the list of 10 nearest neighbor, suppose(Class A=4, B=3, C=3) so new point will be assigned to A as of maximum frequency. 

In [6]:
# define a function that will evaluate a model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

In [8]:
import librosa, IPython
import librosa.display
file = 'MUSIC/cut2/00/00.00000.wav'
signal, sr = librosa.load(file , sr = 22050) 
IPython.display.Audio(signal, rate=sr)

In [9]:

directory = 'MUSIC/cut2'
f = open("my.dat", "wb")
i = 0

for folder in os.listdir(directory):
    if folder == ".DS_Store":
        continue
    i += 1
    if i == 11:
        break
    for file in os.listdir(os.path.join(directory, folder)):
        try:
            file_path = os.path.join(directory, folder, file)
            (rate, sig) = wav.read(file_path)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False, nfft=1024)
            covariance = np.cov(np.transpose(mfcc_feat))
            mean_matrix = np.mean(mfcc_feat, axis=0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)

f.close()


* after the cell run we will be having a my.dat file as a collection of different files irrespective of a jonar.
* we pich each file from genre and calculate rate and signature.
* what we have done is, we find the imp feature as mfcc(mel frequency celestral coefficients) then we find the feature and then we dump the feature into my.dat file.
* Now we have a dataset, so we will split into train and test set

In [10]:
#split dataset into train and test set
dataset = []

def loadDataset(filename, split, trset, teset):
    with open('my.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

* we will load the dataset again to split it, we take 75% as train and 25% as test data.
* we split the dataset based on split value, because directly splitting will not give as good result because we want the random sample which is the mix of all the jonars.

In [11]:
# Make the prediction using KNN(K nearest Neighbors)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.5836653386454184
